In [28]:
!pip install tensorflow

In [114]:
import os
import shutil
import random

# Paths
source_dir = r"C:\Users\YASHRAJ\OneDrive\文档\Skribix-copy\skribix_v2\sketches"
dest_dir = r"C:\Users\YASHRAJ\Last_dataset\normalised"

# Ensure the destination directory exists
os.makedirs(dest_dir, exist_ok=True)

# Get all class folders
all_classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

print("Starting dataset normalization and processing...\n")
print(f"Total classes found: {len(all_classes)}\n")

total_images_copied = 0

for idx, class_name in enumerate(sorted(all_classes), start=1):
    class_source_path = os.path.join(source_dir, class_name)
    class_dest_path = os.path.join(dest_dir, class_name)
    os.makedirs(class_dest_path, exist_ok=True)

    images = sorted(os.listdir(class_source_path))
    
    for img_name in images:
        src_path = os.path.join(class_source_path, img_name)
        dest_path = os.path.join(class_dest_path, img_name)
        shutil.copy(src_path, dest_path)
        total_images_copied += 1

    print(f"[{idx}/{len(all_classes)}] Processed '{class_name}' - {len(images)} images copied.")

print("\nDataset processing complete.")
print(f"Total classes processed: {len(all_classes)}")
print(f"Total images copied: {total_images_copied}")

Starting dataset normalization and processing...

Total classes found: 15

[1/15] Processed 'airplane' - 80 images copied.
[2/15] Processed 'book' - 80 images copied.
[3/15] Processed 'cup' - 80 images copied.
[4/15] Processed 'envelope' - 80 images copied.
[5/15] Processed 'fan' - 80 images copied.
[6/15] Processed 'fork' - 80 images copied.
[7/15] Processed 'hat' - 80 images copied.
[8/15] Processed 'key' - 80 images copied.
[9/15] Processed 'laptop' - 80 images copied.
[10/15] Processed 'leaf' - 80 images copied.
[11/15] Processed 'moon' - 80 images copied.
[12/15] Processed 'pizza' - 80 images copied.
[13/15] Processed 't-shirt' - 80 images copied.
[14/15] Processed 'traffic light' - 80 images copied.
[15/15] Processed 'wineglass' - 80 images copied.

Dataset processing complete.
Total classes processed: 15
Total images copied: 1200


In [116]:
import os
import shutil
import random

# Paths
NORMALIZED_PATH = r"C:\Users\YASHRAJ\Last_dataset\normalised"
DATASET_SPLIT_PATH = r"C:\Users\YASHRAJ\Last_dataset\split"

# Define split percentages
TRAIN_RATIO = 0.7
VAL_RATIO = 0.1
TEST_RATIO = 0.2

# Create dataset split directories
for split in ["train", "val", "test"]:
    split_path = os.path.join(DATASET_SPLIT_PATH, split)
    os.makedirs(split_path, exist_ok=True)

print("Starting dataset splitting...\n")
print(f"Split ratios - Train: {int(TRAIN_RATIO*100)}%, Validation: {int(VAL_RATIO*100)}%, Test: {int(TEST_RATIO*100)}%\n")

total_classes = 0
total_train = 0
total_val = 0
total_test = 0

# Iterate through each class
for class_name in sorted(os.listdir(NORMALIZED_PATH)):
    class_source_path = os.path.join(NORMALIZED_PATH, class_name)
    
    if not os.path.isdir(class_source_path):
        continue  # Skip non-directory files

    images = sorted(os.listdir(class_source_path))
    random.shuffle(images)

    train_count = int(len(images) * TRAIN_RATIO)
    val_count = int(len(images) * VAL_RATIO)

    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Update totals
    total_classes += 1
    total_train += len(train_images)
    total_val += len(val_images)
    total_test += len(test_images)

    # Copy files
    for split, split_images in zip(["train", "val", "test"], [train_images, val_images, test_images]):
        split_class_path = os.path.join(DATASET_SPLIT_PATH, split, class_name)
        os.makedirs(split_class_path, exist_ok=True)

        for img_name in split_images:
            src_img_path = os.path.join(class_source_path, img_name)
            dest_img_path = os.path.join(split_class_path, img_name)
            shutil.copy(src_img_path, dest_img_path)

    print(f"[{total_classes}] {class_name}: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test")

# Summary
print("\nDataset splitting complete.")
print(f"Total classes processed: {total_classes}")
print(f"Total images -> Train: {total_train}, Validation: {total_val}, Test: {total_test}")


Starting dataset splitting...

Split ratios - Train: 70%, Validation: 10%, Test: 20%

[1] airplane: 56 train, 8 val, 16 test
[2] book: 56 train, 8 val, 16 test
[3] cup: 56 train, 8 val, 16 test
[4] envelope: 56 train, 8 val, 16 test
[5] fan: 56 train, 8 val, 16 test
[6] fork: 56 train, 8 val, 16 test
[7] hat: 56 train, 8 val, 16 test
[8] key: 56 train, 8 val, 16 test
[9] laptop: 56 train, 8 val, 16 test
[10] leaf: 56 train, 8 val, 16 test
[11] moon: 56 train, 8 val, 16 test
[12] pizza: 56 train, 8 val, 16 test
[13] t-shirt: 56 train, 8 val, 16 test
[14] traffic light: 56 train, 8 val, 16 test
[15] wineglass: 56 train, 8 val, 16 test

Dataset splitting complete.
Total classes processed: 15
Total images -> Train: 840, Validation: 120, Test: 240


In [119]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Paths
DATASET_SPLIT_PATH = r"C:\Users\YASHRAJ\Last_dataset\split"
MODEL_SAVE_PATH = r"C:\Users\YASHRAJ\Last_CNN\CNN_bestepoch_model.h5"

# Parameters
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 15

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load datasets
print("Loading dataset...\n")

train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_SPLIT_PATH, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(DATASET_SPLIT_PATH, "val"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Define CNN model
print("\nBuilding CNN model...\n")

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ModelCheckpoint callback (only best model will be saved)
checkpoint_cb = ModelCheckpoint(
    filepath=MODEL_SAVE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Start training
print("Starting training...\n")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb],
    verbose=1
)

print(f"\nTraining completed. Best model saved to:\n{MODEL_SAVE_PATH}")


Loading dataset...

Found 840 images belonging to 15 classes.
Found 120 images belonging to 15 classes.

Building CNN model...

Starting training...

Epoch 1/15
27/27 [==============================] - ETA: 0s - loss: 4.0159 - accuracy: 0.1119
Epoch 1: val_accuracy improved from -inf to 0.39167, saving model to C:\Users\YASHRAJ\Last_CNN\CNN_bestepoch_model.h5
27/27 [==============================] - 20s 698ms/step - loss: 4.0159 - accuracy: 0.1119 - val_loss: 2.3917 - val_accuracy: 0.3917
Epoch 2/15
27/27 [==============================] - ETA: 0s - loss: 1.6875 - accuracy: 0.4869
Epoch 2: val_accuracy improved from 0.39167 to 0.51667, saving model to C:\Users\YASHRAJ\Last_CNN\CNN_bestepoch_model.h5
27/27 [==============================] - 19s 685ms/step - loss: 1.6875 - accuracy: 0.4869 - val_loss: 1.6615 - val_accuracy: 0.5167
Epoch 3/15
27/27 [==============================] - ETA: 0s - loss: 0.9924 - accuracy: 0.6845
Epoch 3: val_accuracy improved from 0.51667 to 0.66667, saving mo

In [122]:
# Load test data
print("\nLoading test dataset...\n")

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(DATASET_SPLIT_PATH, "test"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Evaluate model
print("Evaluating model on test data...\n")

test_loss, test_acc = model.evaluate(test_generator, verbose=1)

# Output results
print("\nTest Evaluation Results")
print("------------------------")
print(f"Test Accuracy : {test_acc * 100:.2f}%")
print(f"Test Loss     : {test_loss:.4f}")



Loading test dataset...

Found 240 images belonging to 15 classes.
Evaluating model on test data...

8/8 [==============================] - 2s 192ms/step - loss: 2.7733 - accuracy: 0.6667

Test Evaluation Results
------------------------
Test Accuracy : 66.67%
Test Loss     : 2.7733
